# **Importing Libraries**

In [ ]:
import numpy as np
from os import listdir
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from numpy import expand_dims
from matplotlib import pyplot
from numpy import squeeze
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from deblurgan.model import generator_model

# **Image Loading Function**

In [ ]:
def load_image(filename, size=(256, 256)):
  # load image with the preferred size
  pixels = load_img(filename, grayscale = False, target_size=size)
  
  # convert to numpy array
  pixels = img_to_array(pixels)
  
  # scale from [0,255] to [-1,1]
  pixels = (pixels - 127.5) / 127.5
  
  # reshape to 1 sample
  pixels = expand_dims(pixels, 0)
  
  return pixels

In [ ]:
# Load the model
model = generator_model()
print("model loaded")

# load weights
model.load_weights('../weights/generator_3760.h5')

# load source image
test_dir = '../Images/Test/A/'
comp_dir = '../Images/Test/B/'

img_num = listdir(test_dir)

avg_psnr = []
avg_ssim = []

for i in img_num:
	src_image = load_image(test_dir + i)
	comp_image = load_image(comp_dir + i)
	#print('Loaded Image: ', src_image.shape)
	
	# generate image from source
	gen_image = model.predict(src_image)
	#print('image generated ', gen_image.shape)
	
	# metrics before scaling
	ssim = ssim(gen_image[0], comp_image[0], multichannel = True)
	psnr = psnr(comp_image[0], gen_image[0])  
	avg_psnr.append(psnr)
	avg_ssim.append(ssim) 
  
	# scale from [-1,1] to [0,1]
	comp_image[0] = (comp_image[0] - np.amin(comp_image[0])) / (np.amax(comp_image[0])-np.amin(comp_image[0]))
	gen_image[0] = (gen_image[0] - np.amin(gen_image[0])) / (np.amax(gen_image[0])-np.amin(gen_image[0]))
	
	# plot the image
	pyplot.imshow(gen_image[0].squeeze(), cmap = 'gray')
	pyplot.title('ssim: ' + str(ssim) + '\n' + 'psnr: ' + str(psnr))
	#pyplot.axis('off')
	
	# save the image
	filename = '../Images/Generated/genrated_' + i
	pyplot.imsave(filename, gen_image[0])
	pyplot.show()
	#pyplot.close()

In [ ]:
print("Average PSNR: ", np.mean(avg_psnr))
print("Average SSIM: ", np.mean(avg_ssim))